### Import package

In [166]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

pd.set_option('future.no_silent_downcasting', True)

### Read in files

In [167]:
def read_protein_file(file_path):
    with open(file_path, "r") as file:
        # Read the entire contents of the file
        seq = ""
        str = ""

        sequences = []
        strings = []
        count = 0
        for line in file:
            if count < 8:
                pass
            count += 1

            line = line.strip()
            if line == "<>":
                seq = ""
                str = ""
            elif line == "<end>" or line=="end":
                sequences.append(seq)
                strings.append(str)
            else:
                letters = line.split(" ")
                if len(letters) == 2:
                    seq += letters[0]
                    str += letters[1] if letters[1] != "_" else "c"
        return sequences, strings


In [168]:
train_seq, train_str = read_protein_file("data/protein-secondary-structure.train")
test_seq, test_str = read_protein_file("data/protein-secondary-structure.test")

### NOTE TO SELF
Create sliding windows

Note. we need to split the sequences up, but since our test input is of different lengths, what do we do then?

When your test data contains protein sequences of different lengths compared to your training data, you need to handle this discrepancy appropriately. Here are a few strategies you can consider:

Padding:

Pad shorter sequences in the test data to match the length of the longest sequence in your training data.
You can pad the sequences with a special token (e.g., all zeros) so that the neural network can recognize them as padding.
During inference, you would need to trim the predictions to the original length of the test sequence.
Dynamic Input Shape:

Modify your neural network architecture to accept variable-length input sequences.
Use techniques like masking to handle variable-length sequences effectively.
Many deep learning frameworks, such as TensorFlow and PyTorch, support dynamic input shapes and masking.
Bucketing or Binning:

Group sequences of similar lengths together and pad each group separately to the maximum length within that group.
This approach reduces the amount of padding required, leading to more efficient training.
You can organize your test data into buckets or bins based on sequence lengths and process each bucket separately during evaluation.
Batch Processing:

While processing batches during testing, group sequences of similar lengths together.
This allows you to minimize padding within each batch, improving computational efficiency.

One hot encoding the sequence

Encoding Amino Acids:

First, you need to encode each amino acid into a numerical representation.
Common encoding methods include one-hot encoding and embedding.
In one-hot encoding, each amino acid is represented as a binary vector where only one element is 1 (indicating the presence of that amino acid).
In embedding, each amino acid is mapped to a low-dimensional vector space.
Padding:

After encoding amino acids, you can proceed with padding sequences as discussed earlier.
Pad the sequences to match the length of the longest sequence in your training data.

PERFORM NEURAL NETWORK??? use RNN or CNN (keras, tensorflow, pytorch)

https://www.pnas.org/doi/epdf/10.1073/pnas.86.1.152

sliding window = 17

### Create sliding windows

In [170]:
def split_based_on_windows(data_seq, data_str, W=17):
    all_sequences = []
    all_strings = []
    for i in range(len(data_seq)):
        sequences = [data_seq[i][j:j+W] for j in range(0, len(data_seq[i]), W)]
        strings = [data_str[i][j:j+W] for j in range(0, len(data_str[i]), W)]
        # PADDING
        if len(sequences[-1]) != W:
            sequences[-1] = sequences[-1] + "X"*((W-len(sequences[-1])))
        if len(strings[-1]) != W:
            strings[-1] = strings[-1] + "X"*((W-len(strings[-1])))

        all_sequences += sequences
        all_strings += strings

    train_df = pd.DataFrame({"sequence": all_sequences, "string": all_strings})
    return train_df


In [172]:
train_df = split_based_on_windows(train_seq, train_str)
test_df = split_based_on_windows(test_seq, test_str)

### One hot encode the data

In [174]:
def get_matrix_from_letter_cols(letters, df):
    all_columns = []
    for i in letters:
        columns = [str(j)+"_"+i for j in range(0, 17)]
        all_columns += columns

    template = pd.DataFrame(columns=all_columns, dtype=bool)
    unique_columns = template.columns.difference(df.columns)

    one_hot_encoded_full = pd.concat([df, template[unique_columns]], axis=1).fillna(False)
    ohe_df = one_hot_encoded_full.reindex(sorted(one_hot_encoded_full.columns), axis=1).astype(int)
    return ohe_df.values

In [175]:
def get_ohe_matrix(df):
    X_letters = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "X", "Y"]
    Y_letters = ['c', 'e', 'h', 'X']

    by_pos_df = df['sequence'].apply(lambda x:pd.Series(list(x)))
    ohe_df_X = pd.get_dummies(by_pos_df)
    ohe_X = get_matrix_from_letter_cols(X_letters, ohe_df_X)

    by_pos_df = df['string'].apply(lambda x:pd.Series(list(x)))
    ohe_df_Y = pd.get_dummies(by_pos_df)
    ohe_Y = get_matrix_from_letter_cols(Y_letters, ohe_df_Y)

    return ohe_X, ohe_Y

In [176]:
train_ohe_X, train_ohe_Y = get_ohe_matrix(train_df)
test_ohe_X, test_ohe_Y = get_ohe_matrix(test_df)

### Define the neural network

In [177]:
model = models.Sequential([
    layers.Dense(units=train_ohe_Y.shape[1],input_shape=(train_ohe_X.shape[1],), activation="relu"),
    layers.Dense(train_ohe_Y.shape[1], activation='sigmoid')

])

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer= "adam", metrics=['accuracy'])

# Train the model
model.fit(train_ohe_X, train_ohe_Y, epochs=10, batch_size=32, validation_split=0.2)
predictions = model.predict(test_ohe_X)
output = np.where(predictions > 0.5, 1, 0)
output

Epoch 1/10


24/24 [==============================] - 1s 7ms/step - loss: 71.3121 - accuracy: 0.0187 - val_loss: 70.4473 - val_accuracy: 0.0160
Epoch 2/10
24/24 [==============================] - 0s 2ms/step - loss: 69.9108 - accuracy: 0.0080 - val_loss: 69.8063 - val_accuracy: 0.0000e+00
Epoch 3/10
24/24 [==============================] - 0s 2ms/step - loss: 71.0182 - accuracy: 0.0040 - val_loss: 72.9486 - val_accuracy: 0.0000e+00
Epoch 4/10
24/24 [==============================] - 0s 2ms/step - loss: 76.5235 - accuracy: 0.0000e+00 - val_loss: 79.6916 - val_accuracy: 0.0000e+00
Epoch 5/10
24/24 [==============================] - 0s 3ms/step - loss: 84.2313 - accuracy: 0.0000e+00 - val_loss: 87.3715 - val_accuracy: 0.0000e+00
Epoch 6/10
24/24 [==============================] - 0s 2ms/step - loss: 92.7112 - accuracy: 0.0000e+00 - val_loss: 94.8532 - val_accuracy: 0.0000e+00
Epoch 7/10
24/24 [==============================] - 0s 2ms/step - loss: 101.2570 - accuracy: 0.0027 - val_loss: 102.6823 - val_

array([[0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1]])

### Convert the prediction back

In [231]:
def convert_mat_back_to_seq(mat):
    letters = ['c', 'e', 'h', 'X']

    all_columns = []
    for i in letters:
        columns = [str(j)+"_"+i for j in range(0, 17)]
        all_columns += columns

    template = pd.DataFrame(mat, columns=all_columns, dtype=bool)

    strings = []
    for i in range(mat.shape[0]):
        string = ""
        for j in range(mat.shape[1]//len(letters)):
            for k in letters:
                if template[f"{j}_{k}"].iloc[i]:
                    string+=k
                    break
        strings.append(string)
    return strings

In [232]:
pred_sequences = convert_mat_back_to_seq(output)


190


In [235]:
test_df['predict'] = pred_sequences

In [236]:
test_df.head()

,sequence,string,predict
0,ENLKLGFLVKQPEEPWF,cccccceeeccccccch,ecccccccccccccccc
1,QTEWKFADKAGKDLGFE,hhhhhhhhhhhhhcccc,ecccccccccccccccc
2,VIKIAVPDGEKTLNAID,cceeeccchhhhhhhhh,ecccccccccccccccc
3,SLAASGAKGFVICTPDP,hhhhccccccccccccc,ecccccccccccccccc
4,KLGSAIVAKARGYDMKV,cccchhhhhhhhhcccc,ecccccccccccccccc
